# Dataset Construction

In [9]:
import numpy as np
import pandas as pd
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import torch

class StaticSP500DatasetLoader(object):
    def __init__(self, correlation_type='pearsons'):
        self._read_csv(correlation_type)

    def _read_csv(self, correlation_type):
        self._correlation_matrix = np.fromfile(f's&p500_{correlation_type}.csv', sep=',')
        N = int(np.sqrt(len(self._correlation_matrix)))
        self._correlation_matrix = self._correlation_matrix.reshape(N, N)
        _correlation_threshold = 0.9
        self._correlation_matrix[self._correlation_matrix < _correlation_threshold] = 0
        
        df = pd.read_csv('s&p500.csv')
        df = df.set_index('Date')
        data = torch.from_numpy(df.to_numpy()).to(torch.float32)
        ratios = torch.zeros_like(data)
        # calculate daily return ratio
        for d in range(1, data.size(0)):
            ratios[d-1] = (data[d] - data[d-1]) / data[d-1]
        # skip the first day which cannot calculate daily return ratio
        # and round data size to nearest multiple of batch_size
        days_in_quarter = 64
        num_quarters = data.size(0) // days_in_quarter
        num_days = num_quarters * days_in_quarter
        ratios = ratios[:num_days]
        self._dataset = ratios.numpy()

    def _get_edges(self):
        self._edges = np.array(self._correlation_matrix.nonzero())

    def _get_edge_weights(self):
        self._edge_weights = self._correlation_matrix[self._correlation_matrix > 0]

    def _get_targets_and_features(self):
        stacked_target = self._dataset
        self.features = [
            stacked_target[i : i + self.lags, :].T
            for i in range(stacked_target.shape[0] - self.lags)
        ]
        self.targets = [
            stacked_target[i + self.lags, :].T
            for i in range(stacked_target.shape[0] - self.lags)
        ]

    def get_dataset(self, lags: int) -> StaticGraphTemporalSignal:
        """Returning the data iterator.

        Args types:
            * **lags** *(int)* - The number of time lags.
        Return types:
            * **dataset** *(StaticGraphTemporalSignal)*
        """
        self.lags = lags
        self._get_edges()
        self._get_edge_weights()
        self._get_targets_and_features()
        dataset = StaticGraphTemporalSignal(
            self._edges, self._edge_weights, self.features, self.targets
        )
        return dataset

In [11]:
from torch_geometric_temporal.signal import temporal_signal_split
import torch_geometric

device = 'cpu'

loader = StaticSP500DatasetLoader()

lags = 64

dataset = loader.get_dataset(lags)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

# RGCN

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [14]:
from tqdm import tqdm

model = RecurrentGCN(node_features = lags).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(100)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x.to(device), snapshot.edge_index.to(device), snapshot.edge_attr.to(device))
        cost = cost + torch.mean((y_hat-snapshot.y.to(device))**2)
    cost = cost / (time+1)
    print(f'Epoch {epoch}, MSE: {cost.item()}')
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 0, MSE: 0.006118872202932835


  0%|          | 1/200 [00:07<25:39,  7.74s/it]

Epoch 1, MSE: 0.003357746172696352


  1%|          | 2/200 [00:14<24:31,  7.43s/it]

...

Epoch 19, MSE: 0.0005290175904519856


 10%|█         | 20/200 [02:24<21:38,  7.21s/it]

Epoch 20, MSE: 0.00047033329610712826


 10%|█         | 21/200 [02:36<22:15,  7.46s/it]


KeyboardInterrupt: 

In [ ]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))